## Abstract

In this work we will to take a look at a data visualization using Python and the Titanic dataset. It's not intended to be the most accurate Titanic dataset analysis about Titanic, it's a project resultant of the course Data Analyst Nanodegree by Udacity,however it is a serious project, so I believe it will be interesting to demonstrate the process to generate a data visualization and Make some inferences about the data.

## Introdução

Iremos trabalhar com um conjunto dados contendo dados demográficos e informações de 891 dos 2224 passageiros e tripulantes a bordo do Titanic. É possível encontrar mais informações sobre este conjunto de dados no [site do Kaggle] (https://www.kaggle.com/c/titanic/data).

O trabalho terá como "norte" 3 questões que nós direcionarão na exploração dos dados e no desenvolvimento da visualização. As questões são:

1. Quantidade de pessoas por sexo.
2. Quantidade de crianças, adultos e idosos, independente do sexo.
3. Sexo e idade maioria entre os sobreviventes.

## Preparação dos dados

### Obtendo os dados

#### Imports

Declaração de todos os importes do projeto.

In [2]:
import pandas
import numpy

#### Lendo o arquivo

Lê o arquivo que contém o conjunto de dados e verifica superficialmente se os dados foram carregados corretamente.

In [3]:
titanic_dataframe = pandas.read_csv("data/titanic_data.csv")

#Display total of rows in dataset
print "Total rows = "+str(len(titanic_dataframe))
print ""

#Print only the 5 first rows
print "Preview:"
titanic_dataframe.head(5)

Total rows = 891

Preview:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Limpeza de dados

O processo de limpeza será executado para cada questão que esse trabalho se propõe a explorar, em outras palavras, será um processo iterativo durante as fases de investigação e exploração.

## Investigação e Exploração

De agora em diante começaremos a investigação e exploração dos dados tendo como guia as 3 questões levantadas na introdução.

### Quantidade de pessoas por sexo.

Para responder essa questão nós iremos analisar o nosso conjunto de dados para descobrir se existe algum tipo de dado que possa ser utilizado para nos ajudar na resolução dessa questão.

Ao analisar o conjunto de dados é possível observar que existe uma coluna nomeada como *Sex*, nela aparentemente contém o sexo para cada passageiro da lista, digo aparentemente pois pode existe algum valor inválido ou a ausência de valor para algum registro(entenda passageiro).

Então a primeira coisa que iremos fazer é saber se todos os dados satisfazem as opções *male* ou *female*, para realizar essa tarefa iremos listar todos os dados que não são *male* ou *female*

In [8]:
#verifica se existe algum erro nos valores da coluna Sex
count_errors = 0
for sex in titanic_dataframe['Sex']:
    if sex!="female" and sex!="male":
        count_errors += 1

count_errors 


0

A verificação retornou 0 erros, ou seja, todos os dados da coluna *Sex* do nosso dataframe estão corretos para o nosso objetivo, não precisando de correções ou intervenções, sendo assim seguiremos em frente para satisfazer a questão levantada.

In [18]:
males_count = titanic_dataframe['Sex'].value_counts()['male']
print "Quantidade de homens: "+str(males_count)

females_count = titanic_dataframe['Sex'].value_counts()['female']
print "Quantidade de mulheres: "+str(females_count)

print "Total de Passageiros: "+str(males_count+females_count)
    

Quantidade de homens: 577
Quantidade de mulheres: 314
Total de Passageiros: 891
